In [41]:
import pandas as pd
import numpy as np

### Preprocessing for all possible TABLES

In [42]:
df = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")

In [43]:
import pandas as pd
from pandas_profiling import ProfileReport
profile = ProfileReport(df)

In [44]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [45]:
def fun(x, mode): 
    altitud = str(x).split('-')
    if altitud[0] == "nan":
        if mode == "nan_1":
            return -1
        return np.nan
    if mode == "nan_1":
            return x
    if mode == "min":
        return int(altitud[0])
    if mode == "diff":
        if len(altitud) == 1:
            return 0
        else:
            return int(altitud[1])-int(altitud[0])

df["ALTITUD_MIN"] = df["ALTITUD"].apply(fun, args=("min",))
df["ALTITUD_DIF"] = df["ALTITUD"].apply(fun, args=("diff",))
df["ALTITUD"] = df["ALTITUD"].apply(fun, args=("nan_1",))

In [46]:
df_estacion_gb_mean = df.groupby(by="ID_ESTACION").mean()
df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)

C:\Users\xavid\anaconda3\lib\site-packages\pandas\core\algorithms.py:968: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  warn(f"Unable to sort modes: {err}")
C:\Users\xavid\AppData\Local\Temp\ipykernel_19260\3219566741.py:2: FutureWarning: ['SUPERFICIE', 'PRODUCCION'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)


In [47]:
df_estacion_gb_mean.reset_index(inplace=True)
df_estacion_gb_mode.reset_index(inplace=True)

In [48]:
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "0", "ALTITUD"] = 390
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "4", "ALTITUD"] = 520

In [49]:
for i in df["ID_ESTACION"].unique():
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_MIN"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_MIN"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_DIF"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_DIF"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD"] = df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == i,"ALTITUD"].values[0]

In [50]:
df.loc[df["MODO"]=="1", "MODO"]  = "0"
df.loc[df["MODO"]=="2", "MODO"]  = "1"

In [51]:
df = df.drop(index=df[df["CAMPAÑA"] == "22"].index)

### 1 - Table without SUPERFICIE

In [185]:
df_temporal = df.drop(axis = 1, columns = ["SUPERFICIE"])

In [186]:
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])
df_encoded = encoder.fit_transform(df_temporal)
encoder = ce.OrdinalEncoder(cols = ["CAMPAÑA"])
df_encoded = encoder.fit_transform(df_temporal)

In [169]:
df_temporal.to_csv("D_T_1.csv")
df_encoded.to_csv("D_T_1_encoded.csv")

### 2 - Table without CAMPAÑA and SUPERFICIE

In [187]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA", "SUPERFICIE"])

In [188]:
import category_encoders as ce
encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])

df_encoded = encoder.fit_transform(df_temporal)

In [172]:
df_temporal.to_csv("D_T_2.csv")
df_encoded.to_csv("D_T_encoded_2.csv")

### 3 - Table without CAMPAÑA

In [12]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA"])

In [14]:
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])

df_encoded = encoder.fit_transform(df_temporal)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [15]:
train = df_encoded[(df_encoded["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0)]
to_predict = df_encoded[~((df_encoded["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0))]

In [16]:
X = train.drop(axis = 1, columns = ["SUPERFICIE"])
y = train["SUPERFICIE"]
to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])

In [17]:
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

normalizer = StandardScaler()
norm_X = normalizer.fit_transform(X)
norm_to_predict = normalizer.transform(to_predict)

In [247]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor


model = {"model_name": "SVC", "model": MLPRegressor(), "params": {'hidden_layer_sizes':[(100),(1000,1000,1000)],
                                                                  'learning_rate':["constant", "adaptive"],}}

grid_search = GridSearchCV(model["model"], model["params"], scoring="r2")
grid_search.fit(norm_X, y)

C:\Users\xavid\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


GridSearchCV(estimator=MLPRegressor(),
             param_grid={'hidden_layer_sizes': [100, (100, 100, 100)],
                         'learning_rate': ['constant', 'adaptive']},
             scoring='r2')

In [248]:
print(grid_search.best_params_)
df = pd.DataFrame(grid_search.cv_results_)
df

{'hidden_layer_sizes': (100, 100, 100), 'learning_rate': 'adaptive'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,9.451171,3.131344,0.005322,0.000376,100,constant,"{'hidden_layer_sizes': 100, 'learning_rate': '...",0.774989,0.799511,0.870512,0.590862,0.665164,0.740207,0.099641,4
1,9.308895,1.867798,0.004107,0.000587,100,adaptive,"{'hidden_layer_sizes': 100, 'learning_rate': '...",0.784408,0.759720,0.861517,0.754148,0.586562,0.749271,0.089934,3
2,8.366059,0.976483,0.005138,0.000371,"(100, 100, 100)",constant,"{'hidden_layer_sizes': (100, 100, 100), 'learn...",0.891863,0.824670,0.908488,0.864096,0.617077,0.821239,0.105953,2
3,8.098244,1.237529,0.004801,0.000393,"(100, 100, 100)",adaptive,"{'hidden_layer_sizes': (100, 100, 100), 'learn...",0.821207,0.866981,0.892721,0.871692,0.829611,0.856443,0.026912,1


In [19]:
from sklearn.neural_network import MLPRegressor
modelMLPR = MLPRegressor(hidden_layer_sizes=(1000, 1000, 1000), learning_rate="adaptive")
modelMLPR.fit(norm_X, y)
predicted = modelMLPR.predict(norm_to_predict)

C:\Users\xavid\AppData\Local\Temp\ipykernel_14836\1973200238.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["SUPERFICIE"] = predicted


In [39]:
final_predict = df[~((df["PRODUCCION"] != 0) & (df["SUPERFICIE"] != 0))]
final_predict["SUPERFICIE"] = predicted
final_train = df[(df["PRODUCCION"] != 0) & (df["SUPERFICIE"] != 0)]

C:\Users\xavid\AppData\Local\Temp\ipykernel_14836\193630758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_predict["SUPERFICIE"] = predicted


In [40]:
final_df = pd.concat([final_predict, final_train])

In [49]:
final_df.to_csv("first_model/datasets/D_T_3.csv")

### gradientboost scikit

In [52]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA"])

In [53]:
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD", "COLOR", "TIPO", "MODO"])

df_temporal = encoder.fit_transform(df_temporal)

In [54]:
train = df_encoded[(df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0)]
to_predict = df_encoded[~((df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0))]

In [55]:
X = train.drop(axis = 1, columns = ["SUPERFICIE"])
y = train["SUPERFICIE"]
to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])

In [161]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor


model = {"model_name": "SVC", "model": GradientBoostingRegressor(), "params": {'learning_rate':[0.1,0.03,0.001],
                                                                              'n_estimators': [50, 100, 200],
                                                                               'criterion': ["friedman_mse", "squared_error"],
                                                                               'max_depth': [5,10], 
                                                                               'min_samples_leaf':[2,10,20]}
                                                                              }

grid_search = GridSearchCV(model["model"], model["params"], scoring="r2", n_jobs = -1)
grid_search.fit(X, y)

KeyboardInterrupt: 

In [ ]:
print(grid_search.best_params_)
df = pd.DataFrame(grid_search.cv_results_)
df

In [154]:
model = GradientBoostingRegressor()
model.fit(X,y)

GradientBoostingRegressor()

In [160]:
y

6476    3.7503
6477    7.3735
6478    7.6200
6480    7.1657
6481    3.5000
         ...  
8521    3.6800
8522    4.2500
8523    4.1600
8524    4.7500
8525    1.4620
Name: SUPERFICIE, Length: 2017, dtype: float64

In [155]:
predicted = model.predict(to_predict)

In [156]:
final_predict = df[~((df["PRODUCCION"] != 0) & (df["SUPERFICIE"] != 0))]
final_predict["SUPERFICIE"] = predicted
final_train = df[(df["PRODUCCION"] != 0) & (df["SUPERFICIE"] != 0)]

C:\Users\xavid\AppData\Local\Temp\ipykernel_14836\193630758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_predict["SUPERFICIE"] = predicted


In [157]:
final_df = pd.concat([final_predict, final_train])

In [158]:
final_df[final_df["ID_FINCA"] == "88928"]

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
1146,14,88928,239,6,700,40,1,0,1,4.895046,45290.0,700.0,0.0
1147,14,88928,239,6,700,52,1,0,1,2.276647,14909.4,700.0,0.0
2262,15,88928,239,6,700,40,1,0,1,4.895046,44620.0,700.0,0.0
2263,15,88928,239,6,700,52,1,0,1,2.276647,15820.2,700.0,0.0
3340,16,88928,239,6,700,40,0,0,1,2.351718,15040.0,700.0,0.0
3341,16,88928,239,6,700,40,1,0,1,5.026932,38100.0,700.0,0.0
3342,16,88928,239,6,700,52,1,0,1,2.390222,19354.5,700.0,0.0
4358,17,88928,239,6,700,40,1,0,1,5.026932,37620.0,700.0,0.0
4359,17,88928,239,6,700,52,1,0,1,2.276647,14978.7,700.0,0.0
5419,18,88928,239,6,700,40,1,0,1,6.366249,57440.0,700.0,0.0


In [213]:
#df = df.astype({"CAMPAÑA": "category", "ID_FINCA": "category", "ID_ZONA": "category", "ID_ESTACION": "category", "ALTITUD": "VARIEDAD": "category", "MODO": "category", "TIPO": "category", "COLOR": "category"})

In [56]:
import category_encoders as ce
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD", "COLOR", "TIPO", "MODO"])

In [57]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA"])
df_encoded = encoder.fit_transform(df_temporal)
train = df_encoded[(df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0)]
to_predict = df_encoded[~((df["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0))]
X = train.drop(axis = 1, columns = ["SUPERFICIE"])
y = train["SUPERFICIE"]
to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])



In [ ]:
"""from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

normalizer = StandardScaler()
norm_X = normalizer.fit_transform(X)
norm_to_predict = normalizer.transform(to_predict)"""

In [66]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
gsc = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"n_estimators": [300, 500, 1000],
                        "learning_rate": [0.15],
                        "max_depth": [40, 60, 100],
                        "min_child_weight": [1],
                        "gamma":[ 0.0],
                        "colsample_bytree":[0.4],},
            cv=5, scoring='r2', verbose=0, n_jobs=-1)

grid_result = gsc.fit(X, y)

In [67]:
print(grid_result.best_params_)
df_grid = pd.DataFrame(grid_result.cv_results_)
df_grid[df_grid["mean_test_score"]>0.86]

{'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.15, 'max_depth': 60, 'min_child_weight': 1, 'n_estimators': 1000}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_gamma,param_learning_rate,param_max_depth,param_min_child_weight,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.631571,0.026786,0.024505,0.005550,0.4,0.0,0.15,40,1,300,"{'colsample_bytree': 0.4, 'gamma': 0.0, 'learn...",0.845011,0.873156,0.868225,0.867454,0.894141,0.869597,0.015659,9
1,1.797871,0.064143,0.019401,0.000800,0.4,0.0,0.15,40,1,500,"{'colsample_bytree': 0.4, 'gamma': 0.0, 'learn...",0.845011,0.873156,0.868225,0.867454,0.894141,0.869597,0.015659,8
2,2.200593,0.031810,0.021606,0.001625,0.4,0.0,0.15,40,1,1000,"{'colsample_bytree': 0.4, 'gamma': 0.0, 'learn...",0.845011,0.873156,0.868225,0.867454,0.894141,0.869597,0.015659,7
3,1.654872,0.024212,0.018104,0.000486,0.4,0.0,0.15,60,1,300,"{'colsample_bytree': 0.4, 'gamma': 0.0, 'learn...",0.845014,0.873504,0.868245,0.867492,0.894146,0.869681,0.015675,5
4,1.813075,0.035935,0.018606,0.001238,0.4,0.0,0.15,60,1,500,"{'colsample_bytree': 0.4, 'gamma': 0.0, 'learn...",0.845014,0.873504,0.868245,0.867492,0.894146,0.869681,0.015675,3
5,2.222707,0.055761,0.022107,0.002243,0.4,0.0,0.15,60,1,1000,"{'colsample_bytree': 0.4, 'gamma': 0.0, 'learn...",0.845014,0.873504,0.868245,0.867492,0.894146,0.869681,0.015675,1
6,1.660686,0.029815,0.020812,0.003293,0.4,0.0,0.15,100,1,300,"{'colsample_bytree': 0.4, 'gamma': 0.0, 'learn...",0.845014,0.873504,0.868245,0.867492,0.894146,0.869681,0.015675,5
7,1.773622,0.023207,0.018906,0.001021,0.4,0.0,0.15,100,1,500,"{'colsample_bytree': 0.4, 'gamma': 0.0, 'learn...",0.845014,0.873504,0.868245,0.867492,0.894146,0.869681,0.015675,3
8,1.691828,0.089754,0.016237,0.001475,0.4,0.0,0.15,100,1,1000,"{'colsample_bytree': 0.4, 'gamma': 0.0, 'learn...",0.845014,0.873504,0.868245,0.867492,0.894146,0.869681,0.015675,1


In [68]:
params_model = grid_result.best_params_
model = xgb.XGBRegressor(**params_model)
model.fit(X,y)
predicted = model.predict(to_predict)

In [69]:
final_predict = df[~((df["PRODUCCION"] != 0) & (df["SUPERFICIE"] != 0))]
final_predict["SUPERFICIE"] = predicted
final_train = df[(df["PRODUCCION"] != 0) & (df["SUPERFICIE"] != 0)]

C:\Users\xavid\AppData\Local\Temp\ipykernel_19260\193630758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_predict["SUPERFICIE"] = predicted


In [70]:
final_df = pd.concat([final_predict, final_train])

In [73]:
final_df[final_df["ID_FINCA"] == "88928"]

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
1146,14,88928,239,6,700,40,1,0,1,4.784074,45290.0,700.0,0.0
1147,14,88928,239,6,700,52,1,0,1,2.200699,14909.4,700.0,0.0
2262,15,88928,239,6,700,40,1,0,1,5.157989,44620.0,700.0,0.0
2263,15,88928,239,6,700,52,1,0,1,2.126570,15820.2,700.0,0.0
3340,16,88928,239,6,700,40,0,0,1,3.096654,15040.0,700.0,0.0
3341,16,88928,239,6,700,40,1,0,1,4.713306,38100.0,700.0,0.0
3342,16,88928,239,6,700,52,1,0,1,2.064521,19354.5,700.0,0.0
4358,17,88928,239,6,700,40,1,0,1,4.542036,37620.0,700.0,0.0
4359,17,88928,239,6,700,52,1,0,1,2.227695,14978.7,700.0,0.0
5419,18,88928,239,6,700,40,1,0,1,4.719661,57440.0,700.0,0.0
